# ItsDangerous package


`itsdangerous` is a Python package to sign cryptographically some data.  
It is used to pass data to an untrusted party and get it back later.

### Installation

`pip install itsdangerous`

### Secret key and salt

Signed tokens are generated from a Signer or a Serializer from a secret key and a salt.  
The secret key is a long string of bytes and must be kept secret (provided as an env param or read from a file but not stored in source code).  
If the secret key is changed, all existing tokens get invalidated.  
The secret key can be generated by `os.urandom()`.

The salt is used in combination to the secret key to reive a unique key for different contexts.  
It does not need to be secret but must be unique between contexts.  
For example if we generate a signed token containing the user name for an activation link, and then we generate another signed token with the same user name for an upgrade account email, we do not want the token from the activation link to be valid for the upgrade.

In [63]:
import os

secret_key = os.urandom(16).hex()      # should normally not be hardcoded
salt       = 'SALT'
data       = {'username': 'bobby123'}

print('Secret     : ', secret_key)

Secret     :  e4464520b6477ff153c13087bb0fcef0


### Serialize/deserialize a token

We can serialize any data into a token with the `dumps()` method of the serializer.  
That token is sent to the untrusted party, then received back, and checked with the `loads()` method of the serializer.


In [64]:
from itsdangerous import Serializer

# Create a serializer from a secret key and salt
serializer = Serializer(secret_key, salt=salt)

# Generate a token for this data that can be sent to the untrusted party
token = serializer.dumps(data)
print('Token      :', token)

# When receiving back the token, confirm it was not altered
decrypted = serializer.loads(token)
print('Decrypted  :', decrypted)

Token      : {"username": "bobby123"}.GlIty28J6tZIsQ2zpBOaAEecbg0
Decrypted  : {'username': 'bobby123'}


If the data was altered or was generated from a different key or salt, the `loads()` method will throw an exception :

In [65]:
# Encrypt data with a salt
serializer  = Serializer(secret_key, salt=salt)
token       = serializer.dumps(data)

# Try to decrypt using a different salt
serializer  = Serializer(secret_key, salt='BBB')
try:
    decrypted   = serializer.loads(token) 
except Exception as e:
    print(type(e).__name__, ' : ', e.message)

BadSignature  :  Signature b'GlIty28J6tZIsQ2zpBOaAEecbg0' does not match


### Serializing data with an expiration

If the serialized data must be valid only for a given period of time (for example a signature for account activation valid only for 30min), we can use the `TimedSerializer` class instead.


In [66]:
from itsdangerous.timed import TimedSerializer
import time
 
serializer = TimedSerializer(secret_key, salt=salt)
token      = serializer.dumps(data)
print('Token     : ', token)

for i in range(5):
    time.sleep(1)
    try:
        decrypted = serializer.loads(token, max_age=3) 
        print('Decrypted : ', decrypted)
    except Exception as e:
        print(type(e).__name__, ' : ', e.message)

Token     :  {"username": "bobby123"}.YQav3w.sUr6n1srAJmex_h1FL-KBzaXRAQ
Decrypted :  {'username': 'bobby123'}
Decrypted :  {'username': 'bobby123'}
Decrypted :  {'username': 'bobby123'}
SignatureExpired  :  Signature age 4 > 3 seconds
SignatureExpired  :  Signature age 5 > 3 seconds


### URL-safe Serializer

Sometimes it is useful to include the generated token as a URL parameter accepting only a specific set of characters.  
In that case we can use the `URLSafeSerializer` or `URLSafeTimedSerializer` classes.  
They generate tokens containing only sequences of letters, numbers, dots, underscores and hyphens.

In [67]:
from itsdangerous.url_safe import URLSafeSerializer, URLSafeTimedSerializer

# URL safe serializer
serializer = URLSafeSerializer(secret_key, salt=salt)
token      = serializer.dumps(data)
print('URL-safe token       : ', token)

decrypted   = serializer.loads(token) 
print('Decrypted            : ', decrypted)

# URL safe serializer with time expiration
serializer = URLSafeTimedSerializer(secret_key, salt=salt)
token      = serializer.dumps(data)
print('Timed URL-safe Token : ', token)

for i in range(5):
    time.sleep(1)
    try:
        decrypted = serializer.loads(token, max_age=3) 
        print('Decrypted            : ', decrypted)
    except Exception as e:
        print(type(e).__name__, ' : ', e.message)

URL-safe token       :  eyJ1c2VybmFtZSI6ImJvYmJ5MTIzIn0.TFpTiPXN9rxYSWiDe58dUSEHlic
Decrypted            :  {'username': 'bobby123'}
Timed URL-safe Token :  eyJ1c2VybmFtZSI6ImJvYmJ5MTIzIn0.YQav5A.5Tlptz0Btfs76pSAPDLotokTZ0A
Decrypted            :  {'username': 'bobby123'}
Decrypted            :  {'username': 'bobby123'}
Decrypted            :  {'username': 'bobby123'}
SignatureExpired  :  Signature age 4 > 3 seconds
SignatureExpired  :  Signature age 5 > 3 seconds
